# Wzorce projektowe w Pythonie

Stopniowe wprowadzenie od prostych idiomów do złożonych schematów projektowych.


## Cele
- zrozumieć rolę wzorców projektowych w dynamicznym języku jak Python
- przećwiczyć implementację wzorców poprzez idiomy i biblioteki standardowe
- wybrać właściwy wzorzec w zależności od problemu


## Wprowadzenie
Wzorce projektowe to potwierdzone rozwiązania typowych problemów architektonicznych.
W Pythonie wiele z nich przyjmuje uproszczoną formę dzięki funkcjom pierwszej klasy,
modułom i dynamicznym atrybutom. Zaczniemy od kreacyjnych idiomów.


## Fabryki i rejestry
Fabryka upraszcza tworzenie obiektów, ukrywając logikę wyboru typu.
Rejestr klas pomaga rozszerzać system bez edycji istniejącego kodu.


In [ ]:
from dataclasses import dataclass

@dataclass
class CsvExporter:
    delimiter: str = ","

    def export(self, rows):
        return "".join(self.delimiter.join(row) for row in rows)

@dataclass
class JsonExporter:
    indent: int = 2

    def export(self, rows):
        import json  # import lokalny, by ograniczyć zależność
        return json.dumps(rows, indent=self.indent)

EXPORTERS = {}

def register_exporter(name: str):
    """Dekorator rejestrujący klasę eksportera pod zadanym kluczem."""
    def wrapper(cls):
        EXPORTERS[name] = cls
        return cls
    return wrapper

@register_exporter("csv")
class RegisteredCsv(CsvExporter):
    pass

@register_exporter("json")
class RegisteredJson(JsonExporter):
    pass

def create_exporter(kind: str, **kwargs):
    cls = EXPORTERS[kind]
    return cls(**kwargs)

rows = [["imie", "wiek"], ["Ada", "28"]]
exporter = create_exporter("csv")
print(exporter.export(rows))


## Wzorzec Strategia
Strategia pozwala wymieniać algorytm w locie. W Pythonie często używamy funkcji
lub obiektów wywoływalnych jako strategii.


In [ ]:
from typing import Callable

# Strategie naliczania podatku jako funkcje

def tax_flat(amount: float) -> float:
    return amount * 0.19  # stała stawka liniowa


def tax_progressive(amount: float) -> float:
    return amount * 0.17 if amount <= 120000 else amount * 0.32


class TaxCalculator:
    def __init__(self, strategy: Callable[[float], float]):
        self.strategy = strategy

    def compute(self, amount: float) -> float:
        return self.strategy(amount)


calc = TaxCalculator(tax_flat)
print(calc.compute(10_000))
calc.strategy = tax_progressive  # zmiana algorytmu w trakcie działania
print(calc.compute(200_000))


## Wzorzec Obserwator
Obserwator pozwala reagować na zdarzenia bez ścisłego sprzężenia klas.
Pythonowe listy wywołań i funkcje anonimowe upraszczają implementację.


In [3]:
class EventBus:
    def __init__(self):
        self._subscribers: dict[str, list] = {}

    def subscribe(self, topic: str, handler):
        handlers = self._subscribers.setdefault(topic, [])
        handlers.append(handler)

    def publish(self, topic: str, payload):
        for handler in self._subscribers.get(topic, []):
            handler(payload)

bus = EventBus()
bus.subscribe("user_registered", lambda data: print("Witamy", data["name"]))
bus.subscribe("user_logout", lambda data: print("Żegnamy", data["name"]))
bus.publish("user_registered", {"name": "Celina"})
bus.publish("user_registered", {"name": "Rafal"})

bus.publish("user_logout", {"name": "Celina"})
bus.publish("user_logout", {"name": "Rafal"})


Witamy Celina
Witamy Rafal
Żegnamy Celina
Żegnamy Rafal


**Podsumowanie:** Wzorce w Pythonie często wyrażamy prostymi idiomami.

**Pytanie kontrolne:** Jakie zalety ma strategia oparta o funkcje zamiast klas?


### 🧩 Zadanie 1
Zaimplementuj wzorzec Polecenie (`Command`), który pozwoli rejestrować działania
w kolejce i wycofywać ostatnie polecenie.


In [ ]:
# Rozwiązanie Zadania 1
from collections import deque

class Command:
    def __init__(self, do, undo, description: str):
        self.do = do
        self.undo = undo
        self.description = description

class CommandProcessor:
    def __init__(self):
        self._history = deque()

    def execute(self, command: Command):
        command.do()
        self._history.append(command)
        print("Wykonano:", command.description)

    def undo_last(self):
        command = self._history.pop()
        command.undo()
        print("Cofnięto:", command.description)

state = {"balance": 0}

cmd_inc = Command(
    do=lambda: state.update(balance=state["balance"] + 100),
    undo=lambda: state.update(balance=state["balance"] - 100),
    description="Zwiększenie środków",
)

processor = CommandProcessor()
processor.execute(cmd_inc)
print(state)
processor.undo_last()
print(state)


### 🧩 Zadanie 2
Stwórz rejestr pluginów, który pozwoli dekoratorem `@plugin("nazwa")`
dodawać nowe funkcje przetwarzające dane, a następnie uruchomić je wszystkie
na przekazanym wejściu.


In [ ]:
# Rozwiązanie Zadania 2
PLUGINS: dict[str, list] = {}

def plugin(name: str):
    def decorator(func):
        PLUGINS.setdefault(name, []).append(func)
        return func
    return decorator

@plugin("sanitize")
def strip_whitespace(text: str) -> str:
    return text.strip()

@plugin("sanitize")
def to_lower(text: str) -> str:
    return text.lower()

def run_plugins(group: str, value: str) -> str:
    result = value
    for func in PLUGINS.get(group, []):
        result = func(result)
    return result

print(run_plugins("sanitize", "  Witaj Świecie  "))
